## 1. Install the twitter library in python

In [4]:
!pip install tweepy

## 2. Install library for JSON

In [5]:
!pip install simplejson

## 3. Install sentiment analysis library

In [4]:
# Whatever library you use
!pip install textblob

     |████████████████████████████████| 645kB 1.0MB/s eta 0:00:01


# Streaming tweets and perform some data analysis


### Setting up and running a streaming crawler

In [2]:
import tweepy
import simplejson as json
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy.streaming import StreamListener
from textblob import TextBlob
 
#Complete with your keys 

consumer_key = 'aCqrloIHCuvw8rX8sON8B551T'
consumer_secret = 'CkkLBIClzyC2w00oHOateou4wxJ2IberPKXWR8WpQEi0T5Gh2s'
access_token = '173960222-v34xiH0gEFqXCoVvpnh3JgMp8xwVukAX61rqazBc'
access_secret = '9ey7wPBUkPJdu2ZxaBoOUr43dmFCj2Fql1MxBCPKEoVFg'
 
auth = OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_secret)
print("Starting code")
print(auth)
 

class MyListener(StreamListener):
    
    def __init__(self, filename, api=None):
        super(StreamListener, self).__init__()
        self.num_tweets = 0
        self.filename = filename

    def on_data(self, data):
        try:
            with open(self.filename, 'a') as f:
                #print("test")
                if json.loads(data).get('place'):
                    print(json.loads(data)['place']['country'])
                    if json.loads(data)['place']['country'].lower() in ['spain', 'españa', 'espana']:
                        f.write(data) # This will store the whole JSON data in the file, you can perform some JSON filters
                        twitter_text = json.loads(data)['text'] # You can also print your tweets here
                        print(twitter_text)
                        self.num_tweets += 1

                # Just to limit the number of tweets collected to check the 
                # program at the beginning, then increase the limit
                if self.num_tweets < 100:
                    return True
                else:
                    print('______________ END ', self.filename)
                    return False
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True

 
    def on_error(self, status):
        print('Error :', status.place)
        return False
    
"""twitter_stream = Stream(auth, MyListener('gobierno2.json'))
twitter_stream.filter(track=["gobierno", "elecciones"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('pp.json'))
twitter_stream.filter(track=["pp"], languages=["es"]) # Add your keywords and other filters


twitter_stream = Stream(auth, MyListener('psoe.json'))
twitter_stream.filter(track=["psoe"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('cat.json'))
twitter_stream.filter(track=["cataluña", "catalunya"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('vox.json'))
twitter_stream.filter(track=["vox"], languages=["es"]) # Add your keywords and other filters

twitter_stream = Stream(auth, MyListener('ciudadanos.json'))
twitter_stream.filter(track=["ciudadanos", "ciutadans"], languages=["es"]) # Add your keywords and other filters
"""
twitter_stream = Stream(auth, MyListener('podemos.json'))
twitter_stream.filter(track=["podemos"], languages=["es"]) # Add your keywords and other filters

print('_______ End _______')



Starting code
Peru
Venezuela
España
Donde están los 🥚🥚 que os gastais aquí.


KeyboardInterrupt: 

### Store the JSON data in a CSV for analysing

In [6]:
import simplejson as json

# Create the CSV file
files = ["gobierno", "cat", "podemos", "pp", "vox", "ciudadanos", "psoe"]

for file in files:
    with open (file + ".csv", 'w', encoding ='utf-8') as csv:
        # Write the title of the columns (features) that you want to store in the CSV file
        csv.write('id,'+'created_at'+'text,'+'country'+'\n')

        # Copy the data from the JSON file
        with open(file + ".json", 'r', encoding ='utf-8') as jsonfile:
            for tweet in jsonfile: 
                data = json.loads(tweet)

                # The int values should be converted to strings
                csv.write(str(data['id'])+',')
                csv.write(str(data['created_at'])+',') 
                csv.write((str(data['text']).replace('\n', "").replace(',', ""))+',') 
                #csv.write(str(data['place']['country'])) 
                csv.write('\n')
            

### Load the previous CSV into pandas

In [13]:
import pandas as pd
tweets = pd.read_csv('MyFile.csv', index_col=0, encoding='ISO-8859-1')
tweets.head(100)

,id,created_attext,country
1215660391851642881,Fri Jan 10 15:43:17 +0000 2020,@kathlee73732050 @bubbaprog Obama ? Sadly your...,United States
1215660890508447744,Fri Jan 10 15:45:16 +0000 2020,@JeffreyGuterman @realDonaldTrump Guterman You...,United States
1215660967301931008,Fri Jan 10 15:45:35 +0000 2020,ððð,United States
1215661022469443585,Fri Jan 10 15:45:48 +0000 2020,@KeepinUpWithCDJ @KathyB80962699 @chefkatebrow...,United States
1215661066690158592,Fri Jan 10 15:45:58 +0000 2020,Good,United States
1215661085262565376,Fri Jan 10 15:46:03 +0000 2020,@BernieBroStar @kurtbardella @BernieSanders @r...,United States
1215661103625199616,Fri Jan 10 15:46:07 +0000 2020,-----&gt; Heres your sign. https://t.co/6thOl0...,United States
1215661104258527232,Fri Jan 10 15:46:07 +0000 2020,@mommamia1217 @Rosie Because he is the best pr...,United States
1215661106691076098,Fri Jan 10 15:46:08 +0000 2020,So he even fucked up his GOP Senators support?...,United States
1215661118611308546,Fri Jan 10 15:46:11 +0000 2020,So he attacks Iran for personal and political ...,United States


### Analysing the polarity of the tweets

In [ ]:
from textblob import TextBlob 


def clean_tweet(tweet): 
        ''' 
        Utility function to clean tweet text by removing links, special characters 
        using simple regex statements. 
        '''
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t]) 
                                    |(\w+:\/\/\S+)", " ", tweet).split()) 
  
def get_tweet_sentiment(tweet): 
    ''' 
    Utility function to classify sentiment of passed tweet 
    using textblob's sentiment method 
    '''
    # create TextBlob object of passed tweet text 
    analysis = TextBlob(self.clean_tweet(tweet)) 
    # set sentiment 
    if analysis.sentiment.polarity > 0: 
        return 'positive'
    elif analysis.sentiment.polarity == 0: 
        return 'neutral'
    else: 
        return 'negative'                               

def featched_tweets(tweets):
    try:
        for tweet in tweets:
            # empty dictionary to store required params of a tweet 
            parsed_tweet = {} 

            # saving text of tweet 
            parsed_tweet['text'] = tweet.text 
            # saving sentiment of tweet 
            parsed_tweet['sentiment'] = get_tweet_sentiment(tweet.text) 

            # appending parsed tweet to tweets list 
            if tweet.retweet_count > 0: 
                # if tweet has retweets, ensure that it is appended only once 
                if parsed_tweet not in tweets: 
                    tweets.append(parsed_tweet) 
            else: 
                tweets.append(parsed_tweet) 

        # return parsed tweets 
        return tweets
        
    except tweepy.TweepError as e: 
            # print error (if any) 
            print("Error : " + str(e))
                               
def main(): 
     
  
    # picking positive tweets from tweets found before
    ptweets = [tweet for tweet in tweets if tweet['sentiment'] == 'positive'] 
    # percentage of positive tweets 
    print("Positive tweets percentage: {} %".format(100*len(ptweets)/len(tweets))) 
    # picking negative tweets from tweets 
    ntweets = [tweet for tweet in tweets if tweet['sentiment'] == 'negative'] 
    # percentage of negative tweets 
    print("Negative tweets percentage: {} %".format(100*len(ntweets)/len(tweets))) 
    # percentage of neutral tweets 
    print("Neutral tweets percentage: {} % \ 
        ".format(100*len(tweets - ntweets - ptweets)/len(tweets))) 
  
    # printing first 5 positive tweets 
    print("\n\nPositive tweets:") 
    for tweet in ptweets[:10]: 
        print(tweet['text']) 
  
    # printing first 5 negative tweets 
    print("\n\nNegative tweets:") 
    for tweet in ntweets[:10]: 
        print(tweet['text']) 
  
if __name__ == "__main__": 
    main()                                
        

### Generating a wordcloud

In [ ]:
!pip install wordcloud

In [ ]:
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import matplotlib.pyplot as plt

text = " ".join(review for review in tweets.text)
print("There are {} words in the combination of all review. ".format(len(text)))

#create stopword list

stopwords = set(STOPWORDS)
stopwords.update(["http","https", "co"])

# generate wordcloud image
wordcloud = WordCloud(stopwords=stopwords,background_color="white").generate(text)

#Display generated image:

plt.figure(fgisize=(10,15))
plt.imshow(worldcloud, interplotation'bilinear')
plt.axis("off")
plt.show()



### Your own analysis

In [ ]:
...